# COLX 563 Lab Assignment 4: Slot filling
## Assignment Objectives

In this lab, you will build an end-to-end system for basic (binary) intent recognition and slot filling in the context of a dialogue system. It is a team assignment, and you have nearly complete freedom with regards to your solution, with a few restrictions mentioned below. **For this lab, you will work with your capstone team.**

## Getting Started

Add imports below.

In [ ]:
from sklearn.metrics import accuracy_score
import nltk
nltk.download('punkt')
!pip install pulp
!pip install torch
!pip install transformers
!pip install BeautifulSoup
!pip install sklearn_crfsuite
from google.colab import drive
drive.mount('/content/gdrive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 14.2 MB 14.1 MB/s 
     |████████████████████████████████| 4.0 MB 9.7 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.4 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement BeautifulSoup (from versions: 3.2.0, 3.2.1, 3.2.2)
ERROR: No matching distribution found for BeautifulSoup
     |████████████████████████████████| 965 kB 8.4 MB/s 
Mounted at /content/gdrive


In [ ]:
#provided code
import os
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW, Adam
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import accuracy_score

For this lab, you'll be working with the MultiWOZ dataset of goal-oriented dialogues (2.2). You can look at the full corpus [here](https://github.com/budzianowski/multiwoz/tree/master/data/MultiWOZ_2.2). It has an impressively detailed annotation involving multiple turns and multiple goals which we have simplified to just the initiating request (first turn) and involving two possible intents and the corresponding slots for those intents. Download the data from [github](https://github.ubc.ca/jungyeul/labs/raw/master/block6/Multiwoz.zip), unzip it into a directory outside of your lab repo and change the path below.

In [ ]:
#provided code
woz_directory = "/content/gdrive/MyDrive/563/Lab4/data/"

In [ ]:
hotel_slot_order = [
    "hotel-area",
    "hotel-internet",
    "hotel-name",
    "hotel-parking",
    "hotel-pricerange",
    "hotel-stars",
    "hotel-type",
]

restaurant_slot_order = [
    "restaurant-area",
    "restaurant-food",
    "restaurant-name",
    "restaurant-pricerange",
]

intent2ind = {"find_hotel": 0, "find_restaurant": 1}
pricerange2ind = {
    "no_mention": 0,
    "cheap": 1,
    "dontcare": 2,
    "expensive": 3,
    "moderate": 4,
}
area2ind = {
    "no_mention": 0,
    "dontcare": 1,
    "centre": 2,
    "east": 3,
    "north": 4,
    "south": 5,
    "west": 6,
}
parking2ind = {"no_mention": 0, "dontcare": 1, "no": 2, "yes": 3}
internet2ind = {"no_mention": 0, "dontcare": 1, "no": 2, "yes": 3}
star2ind = {
    "no_mention": 0,
    "dontcare": 1,
    "0": 2,
    "1": 3,
    "2": 4,
    "3": 5,
    "4": 6,
    "5": 7,
}
type2ind = {"no_mention": 0, "dontcare": 1, "guesthouse": 2}

iob_labels = {"hotel-name", "restaurant-food", "restaurant-name"}

iob_tags = [
    "O",
    "B-HOTEL-NAME",
    "I-HOTEL-NAME",
    "B-RESTAURANT-NAME",
    "I-RESTAURANT-NAME",
    "B-RESTAURANT-FOOD",
    "I-RESTAURANT-FOOD",
]

iob_tags2ind = {tag: i for i, tag in enumerate(iob_tags)}


label2ind = {
    "find": intent2ind,
    "pricerange": pricerange2ind,
    "area": area2ind,
    "parking": parking2ind,
    "internet": internet2ind,
    "stars": star2ind,
    "type": type2ind,
}


In [ ]:
MODEL_NAME = "distilroberta-base"
BATCH_SIZE = 16

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

## Tidy Submission
rubric={mechanics:1}

To get the marks for tidy submission:
- Submit the assignment by filling in this Jupyter notebook with your answers embedded
- Be sure to follow the instructions

## Inspecting the data

Let's look at corresponding pairs of utterances and answers from the training portion of our corpus

In [ ]:
count = 0
with open(woz_directory + "WOZ_train_utt.txt") as f1:
    with open(woz_directory + "WOZ_train_ans.txt") as f2:
        while count < 20:
            print(f1.readline().strip())
            print(f2.readline().strip())
            print("------")
            count += 1

Guten Tag, I am staying overnight in Cambridge and need a place to sleep. I need free parking and internet.
find_hotel|hotel-area=centre|hotel-internet=yes|hotel-parking=yes
------
Hi there! Can you give me some info on Cityroomz?
find_hotel|hotel-name=cityroomz
------
I am looking for a hotel named alyesbray lodge guest house.
find_hotel|hotel-name=alyesbray lodge guest house
------
I am looking for a restaurant. I would like something cheap that has Chinese food.
find_restaurant|restaurant-food=chinese|restaurant-pricerange=cheap
------
I'm looking for an expensive restaurant in the centre if you could help me.
find_restaurant|restaurant-area=centre|restaurant-pricerange=expensive
------
I'm looking for a places to go and see during my upcoming trip to Cambridge.
find_hotel
------
Yeah, could you recommend a good gastropub?
find_restaurant|restaurant-food=gastropub
------
I want to find an expensive restaurant and serves european food. Can i also have the address, phone number and it

The utterances consists of a request for information about either hotels or restaurants. The first part of the answer starts with the intent (either find_restaurant or find_hotel) and then lists the slots that have been filled in based on the utterance. Your goal is to generate this string of intents and slots based purely on the utterance. A few things to note:

* Not all slots are filled in, and sometimes there are no slots filled in at all (but there is always an intent).
* There are a fixed number of slots for each intent, and they always appear in a particular order, when they are filled in
* The slot values sometimes but do not always correspond to what appears in the utterance. For example, a mention of wanting wifi in the request becomes hotel-internet=yes.

We will be evaluating based on exact duplication of the entire output string, so before you start coding a solution, you should look carefully at examples in the training set and make sure you understand all the different components of the output, and how they related to the input utterance. In particular, you should identify the various constituent parts of the task, and judge which are likely to be easy, and which are likely to be more difficult.

## Solution
rubric={accuracy:10,quality:5,efficiency:3}

You will build a system that, when provided with an utterance, predicts the appropriate intent and slots in the format used in the provided answers. This is an open-ended problem and you may solve it however you like, with the following restrictions:

* Your solution should include at least one of token-level prediction models used in Labs 1-3 of this course, i.e. you should make use of a CRF, an LSTM, or a BERT model. You may use multiple models.
* You may use basic NLP tools (tokenizer, POS, parser) and unsupervised resources such as word embeddings, but you should NOT use an existing NER system, or any additional labeled data for this task.
* Your solution should be appropriately decomposed into parts, and documented. This is a complex enough problem that you should have several functions. You may wrap things up into a single class if you like, but you don't have to.
* Use the provided assert to test `dev_predicted`, the output of your complete model on the dev set, you will need to pass the assert to get full accuracy points. 
* Though you may use dev *accuracy* to guide the development of your model, you should not look at either utterances or answers for the dev (or the test) when developing your model. Limit your inspection of the data (e.g. for the purposes of error analysis) to the training set.

Other things to consider:

* You may want to build "standard" (non-sequential) ML classifiers for some aspects of this problem, but you don't have to!
* You may want to use appropriate lexicons. You can build them yourself, or find some.
* Rather than using statistical classifiers, you may want to use rule-based methods to solve some of the problems you're facing.
* You should probably do regular error analysis, some kind of crossvalidation in the training set is a good approach for this, or you can create another (inspectable) internal dev set by splitting up the training set.
* If you're looking for just a little bit more performance, don't forget to tune your hyperparameters!

In [ ]:
class WOZdataset(Dataset):
    """
    A dataset for housing WOZ data, including
    input ids, label dicts, iobs and padding mask
    """

    def __init__(
        self,
        prefix="train",
        tokenizer=tokenizer,
        label2ind=label2ind,
        iob_labels=iob_labels,
        iob_tags2ind=iob_tags2ind,
    ):
        self.prefix = prefix
        self.tokenizer = tokenizer
        self.label2ind = label2ind
        self.iob_labels = iob_labels
        self.iob_tags2ind = iob_tags2ind

        self.ids, self.mask = self.get_inputs()

        if self.prefix != "test":
            self.labels = self.get_labels()
            self.iobs = self.get_iobs()
        else:
            self.labels = torch.zeros(self.ids.shape[0])
            self.iobs = torch.zeros(self.ids.shape[0])

    def get_inputs(self):
        utterances = (
            open(woz_directory + "WOZ_" + self.prefix + "_utt.txt").read().split("\n")
        )
        result = self.tokenizer(utterances, padding="max_length", truncation=True)
        ids = torch.tensor(result.input_ids)
        mask = torch.tensor(result.attention_mask)
        return ids, mask

    def get_labels(self):
        labels = []
        with open(woz_directory + "WOZ_" + self.prefix + "_ans.txt") as f:
            for line in f:
                label_dict = {}
                # 1. Split all the labels in the text file
                splits = line.strip().split("|")
                # 2. Add the intention to the label dictionary
                label_dict["find"] = torch.tensor(self.label2ind["find"][splits[0]])
                # 3. For each other label, see if we have it or not and match it to it's appropriate encoding
                for label in list(self.label2ind.keys())[1:]:
                    for split in splits[1:]:
                        split_label_name, split_label_value = split.split("=")
                        if label in split_label_name:
                            label_value = torch.tensor(
                                self.label2ind[label][split_label_value]
                            )
                            label_dict[label] = label_value
                    if label not in label_dict.keys():
                        # If the label doesn't exist, make sure to give it a default value.
                        label_dict[label] = torch.tensor(
                            self.label2ind[label]["no_mention"]
                        )
                labels.append(label_dict)

        return labels

    def get_iobs(self):
        iobs = []
        with open(woz_directory + "WOZ_" + self.prefix + "_ans.txt") as f:
            for i, line in enumerate(f):
                splits = line.strip().split("|")
                # Initialize all the iobs to O values which are equal to zero
                iob = torch.zeros(self.ids[i].shape, dtype=torch.long)
                for split in splits[1:]:
                    split_label_name, split_label_value = split.strip().split("=")
                    # Make sure it's an iob tag
                    if split_label_name in self.iob_labels:
                        # For loop to match the iob position inside the tokenized input
                        for j in range(len(self.ids[i])):
                            k = 1
                            # strip and lower to make sure it matches
                            span = (
                                self.tokenizer.decode(self.ids[i][j : j + k])
                                .strip()
                                .lower()
                            )
                            # If the first word matches, we keep adding words
                            # until it's equal or doesn't match anymore
                            while span in split_label_value:
                                if span == split_label_value:
                                    # If it's equal, we add the iob tags
                                    iob_tag = "B-" + split_label_name.upper()
                                    iob[j] = self.iob_tags2ind[iob_tag]
                                    if k > 1:
                                        iob_tag = "I-" + split_label_name.upper()
                                        for l in range(1, k):
                                            iob[j + l] = self.iob_tags2ind[iob_tag]
                                    break
                                k += 1
                                span = (
                                    self.tokenizer.decode(self.ids[i][j : j + k])
                                    .strip()
                                    .lower()
                                )
                iobs.append(iob)

        return iobs

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        labels = self.labels[index]
        ids = self.ids[index]
        mask = self.mask[index]
        iobs = self.iobs[index]
        return ids, labels, iobs, mask


In [ ]:
train_iter = WOZdataset()
dev_iter = WOZdataset("dev")
test_iter = WOZdataset("test")

train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dev_iter, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_iter, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
"""
Guten Tag, I am staying overnight in Cambridge and need a place to sleep. I need free parking and internet.
find_hotel|hotel-area=centre|hotel-internet=yes|hotel-parking=yes
------
Hi there! Can you give me some info on Cityroomz?
find_hotel|hotel-name=cityroomz
------
I am looking for a hotel named alyesbray lodge guest house.
find_hotel|hotel-name=alyesbray lodge guest house
------
I am looking for a restaurant. I would like something cheap that has Chinese food.
find_restaurant|restaurant-food=chinese|restaurant-pricerange=cheap
------
I'm looking for an expensive restaurant in the centre if you could help me.
find_restaurant|restaurant-area=centre|restaurant-pricerange=expensive
"""

assert train_iter[0][1] == {
    "find": torch.tensor(intent2ind["find_hotel"]),
    "pricerange": torch.tensor(pricerange2ind["no_mention"]),
    "area": torch.tensor(area2ind["centre"]),
    "parking": torch.tensor(parking2ind["yes"]),
    "internet": torch.tensor(internet2ind["yes"]),
    "stars": torch.tensor(star2ind["no_mention"]),
    "type": torch.tensor(type2ind["no_mention"]),
}

assert (
    tokenizer.decode(train_iter[1][0][torch.where(train_iter[1][2] != 0)])
    .strip()
    .lower()
    == "cityroomz"
)

assert (
    train_iter[1][2][torch.where(train_iter[1][2] != 0)][0]
    == iob_tags2ind["B-HOTEL-NAME"]
    and train_iter[1][2][torch.where(train_iter[1][2] != 0)][1]
    == iob_tags2ind["I-HOTEL-NAME"]
)

assert (
    tokenizer.decode(train_iter[2][0][torch.where(train_iter[2][2] != 0)])
    .strip()
    .lower()
    == "alyesbray lodge guest house"
)

assert (
    train_iter[2][2][torch.where(train_iter[2][2] != 0)][0]
    == iob_tags2ind["B-HOTEL-NAME"]
    and train_iter[2][2][torch.where(train_iter[2][2] != 0)][1]
    == iob_tags2ind["I-HOTEL-NAME"]
)

assert train_iter[3][1] == {
    "find": torch.tensor(intent2ind["find_restaurant"]),
    "pricerange": torch.tensor(pricerange2ind["cheap"]),
    "area": torch.tensor(area2ind["no_mention"]),
    "parking": torch.tensor(parking2ind["no_mention"]),
    "internet": torch.tensor(internet2ind["no_mention"]),
    "stars": torch.tensor(star2ind["no_mention"]),
    "type": torch.tensor(type2ind["no_mention"]),
}

assert (
    tokenizer.decode(train_iter[3][0][torch.where(train_iter[3][2] != 0)])
    .strip()
    .lower()
    == "chinese"
)

assert (
    train_iter[3][2][torch.where(train_iter[3][2] != 0)][0]
    == iob_tags2ind["B-RESTAURANT-FOOD"]
)

assert train_iter[4][1] == {
    "find": torch.tensor(intent2ind["find_restaurant"]),
    "pricerange": torch.tensor(pricerange2ind["expensive"]),
    "area": torch.tensor(area2ind["centre"]),
    "parking": torch.tensor(parking2ind["no_mention"]),
    "internet": torch.tensor(internet2ind["no_mention"]),
    "stars": torch.tensor(star2ind["no_mention"]),
    "type": torch.tensor(type2ind["no_mention"]),
}


In [ ]:
class WOZIOBClassifier(nn.Module):
    def __init__(self, hidden_size, num_tags):
        super(WOZIOBClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_tags = num_tags
        self.classifier = nn.Linear(self.hidden_size, self.num_tags)

    def forward(self, hidden_states):
        out = self.classifier(hidden_states).log_softmax(dim=-1)
        return out


class WOZLabelClassifier(nn.Module):
    def __init__(self, hidden_size, label2ind):
        super(WOZLabelClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.label2ind = label2ind
        self.classifier_dict = dict()

        for label in self.label2ind:
            self.classifier_dict[label] = nn.Linear(
                self.hidden_size, len(self.label2ind[label])
            ).to(DEVICE)

    def forward(self, cls_state):
        out_label_dict = dict()
        for label, classifier in self.classifier_dict.items():
            out_label_dict[label] = classifier(cls_state).log_softmax(dim=-1)
        return out_label_dict


class WOZClassifier(nn.Module):
    def __init__(
        self,
        bert_model_name=MODEL_NAME,
        label2ind=label2ind,
        iob_tags2ind=iob_tags2ind,
        dropout=0.1,
    ):
        super(WOZClassifier, self).__init__()
        self.label2ind = label2ind
        self.iob_tags2ind = iob_tags2ind
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.label_classifier = WOZLabelClassifier(
            self.bert.config.hidden_size, self.label2ind
        )
        self.iob_classifier = WOZIOBClassifier(
            self.bert.config.hidden_size, len(iob_tags2ind)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        hidden_states = self.dropout(
            self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        )
        cls_state = hidden_states[:, 0, :]
        labels = self.label_classifier(cls_state)
        iobs = self.iob_classifier(hidden_states)

        return labels, iobs

    def decode(
        self,
        input_ids,
        attention_mask,
        hotel_slot_order=hotel_slot_order,
        restaurant_slot_order=restaurant_slot_order,
        iob_labels=iob_labels,
        iob_tags=iob_tags,
        tokenizer=tokenizer
    ):
        # The goal of this function is to decode the output
        # of the model to its orginal string format
        encoded_output = []

        # Step 1 : Get the predictions from the model
        hidden_states = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        cls_state = hidden_states[:, 0, :]
        labels = self.label_classifier(cls_state)
        iobs = self.iob_classifier(hidden_states).argmax(dim=-1)

        for label_name in labels:
            labels[label_name] = labels[label_name].argmax(dim=-1)

        lengths = attention_mask.sum(dim=-1)

        # Step 2: Go through the predictions one by one
        for i, length in enumerate(lengths):
            # Step 2.1 : Decode the IOB output by getting only ids
            # that are not equal to zero and saving them for latter.
            iob = iobs[i, :length]
            non_zero_iob_ids = torch.where(iob != 0)
            iob_labels_tag_ids = iob[non_zero_iob_ids]
            iob_labels_strings = dict()
            for iob_label_tag_id, iob_label_token_id in zip(
                iob_labels_tag_ids, input_ids[i, :length][non_zero_iob_ids]
            ):
                current_label = iob_tags[iob_label_tag_id]
                iob_split = current_label[:1]
                tag_split = current_label[2:].lower()
                if iob_split == "B":
                    iob_labels_strings[tag_split] = [iob_label_token_id.item()]
                else:
                    if tag_split in iob_labels_strings.keys():
                        iob_labels_strings[tag_split].append(iob_label_token_id.item())

            # Step 2.2 : Find the intention so that
            # we can look at the labels in the right order.
            intention_id = labels["find"][i]

            enc = ""
            if self.label2ind["find"]["find_hotel"] == intention_id:
                intention_name = "find_hotel"
                slot_order = hotel_slot_order
            elif self.label2ind["find"]["find_restaurant"] == intention_id:
                intention_name = "find_restaurant"
                slot_order = restaurant_slot_order

            # Step 2.3 : Get the values one by one for each label in order.
            enc += intention_name
            for slot in slot_order:
                if slot in iob_labels:
                    if slot in iob_labels_strings:
                        enc += (
                            "|"
                            + slot
                            + "="
                            + tokenizer.decode(torch.tensor(iob_labels_strings[slot]))
                            .strip()
                            .lower()
                        )
                else:
                    slot_name = slot.split("-")[1]
                    predicted_slot_value = labels[slot_name][i]
                    if predicted_slot_value != 0:
                        # Have to use this undirect way to get the key matching the id
                        # Ideally, we should've had a key and id mapping going both ways
                        predicted_slot_value_str = list(
                            self.label2ind[slot_name].keys()
                        )[
                            list(self.label2ind[slot_name].values()).index(
                                predicted_slot_value
                            )
                        ]
                        enc += "|" + slot + "=" + predicted_slot_value_str

            encoded_output.append(enc)

        return encoded_output


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [ ]:
def label_loss_fn(pred_labels, gold_labels):
    cel = nn.CrossEntropyLoss()
    out = 0
    for label in gold_labels:
        out += cel(pred_labels[label], gold_labels[label])

    return out


def iob_loss_fn(pred_iob, gold_iob, mask):
    length = mask.sum()
    cel = nn.CrossEntropyLoss()
    out = cel(pred_iob[:, :length, :].permute(0, 2, 1), gold_iob[:, :length])
    return out


def train_model(dataloader, model, optimizer):
    model.train()
    total_loss = 0

    for inputs, labels, iobs, masks in tqdm(dataloader, total=len(dataloader)):
        model.zero_grad()

        inputs = inputs.to(DEVICE)
        for label_name in labels:
            labels[label_name] = labels[label_name].to(DEVICE)
        iobs = iobs.to(DEVICE)
        masks = masks.to(DEVICE)

        pred_labels, pred_iobs = model(input_ids=inputs, attention_mask=masks)

        label_loss = label_loss_fn(pred_labels, labels)
        iob_loss = iob_loss_fn(pred_iobs, iobs, masks)

        loss = label_loss + iob_loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss


def eval_model(dataloader, model):
    model.eval()

    gold = []
    pred = []

    for inputs, labels, iobs, masks in tqdm(dataloader, total=len(dataloader)):
        inputs = inputs.to(DEVICE)
        masks = masks.to(DEVICE)

        pred_labels, pred_iobs = model(input_ids=inputs, attention_mask=masks)

        for label_name in model.label2ind:
            gold.extend(labels[label_name])
            pred.extend(pred_labels[label_name].argmax(dim=-1).cpu())

        for iob in iobs:
            gold.extend(iob)

        for iob in pred_iobs.argmax(dim=-1).cpu():
            pred.extend(iob)

    return accuracy_score(gold, pred)


def eval_model_label_accuracy(dataloader, model):
    model.eval()

    gold = []
    pred = []

    for inputs, labels, iobs, masks in tqdm(dataloader, total=len(dataloader)):
        inputs = inputs.to(DEVICE)
        masks = masks.to(DEVICE)

        pred_labels, pred_iobs = model(input_ids=inputs, attention_mask=masks)

        for label_name in model.label2ind:
            non_zero_label_ids = torch.where(labels[label_name] != 0)
            gold.extend(labels[label_name][non_zero_label_ids])
            pred.extend(
                pred_labels[label_name].argmax(dim=-1)[non_zero_label_ids].cpu()
            )

    return accuracy_score(gold, pred)


def eval_model_iob_accuracy(dataloader, model):
    model.eval()

    gold = []
    pred = []

    for inputs, labels, iobs, masks in tqdm(dataloader, total=len(dataloader)):
        inputs = inputs.to(DEVICE)
        masks = masks.to(DEVICE)

        lengths = masks.sum(dim=-1)

        pred_labels, pred_iobs = model(input_ids=inputs, attention_mask=masks)

        for iob_gold, iob_pred, length in zip(iobs, pred_iobs.argmax(dim=-1).cpu(), lengths):
            iob_gold = iob_gold[:length]
            iob_pred = iob_pred[:length]
            non_zero_iob_ids = torch.where(iob_gold != 0)
            gold.extend(iob_gold[non_zero_iob_ids])
            pred.extend(iob_pred[non_zero_iob_ids])

    return accuracy_score(gold, pred)


In [ ]:
LEARNING_RATE = 5e-5
EPOCHS = 20


model = WOZClassifier()
model.to(DEVICE)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    print(f"EPOCH: {epoch+1} of {EPOCHS}")
    epoch_loss = train_model(train_dataloader, model, optimizer)
    print(f"loss = {epoch_loss / len(train_dataloader)}")


Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EPOCH: 1 of 20


100%|██████████| 235/235 [02:43<00:00,  1.43it/s]


loss = 1.9529592645929215
EPOCH: 2 of 20


...

100%|██████████| 235/235 [02:48<00:00,  1.40it/s]


loss = 0.1142849826115243
EPOCH: 19 of 20


100%|██████████| 235/235 [02:48<00:00,  1.40it/s]


loss = 0.10677737143445522
EPOCH: 20 of 20


 89%|████████▉ | 210/235 [02:30<00:17,  1.40it/s]

In [ ]:
# model = torch.load('cpkt/woz_full_model.pth')

In [ ]:
eval_model(dev_dataloader, model)

100%|██████████| 26/26 [00:06<00:00,  3.80it/s]


0.9995567934237475

In [ ]:
eval_model_label_accuracy(dev_dataloader, model)

100%|██████████| 26/26 [00:06<00:00,  4.25it/s]


0.9853420195439739

In [ ]:
eval_model_iob_accuracy(dev_dataloader, model)

100%|██████████| 26/26 [00:06<00:00,  4.29it/s]


0.9599236641221374

In [ ]:
# torch.save(model, "cpkt/woz_full_model.pth")

## Report
rubric={raw:2,reasoning:3,writing:1}

Describe your system, and discuss what your thinking about particular choices and any experiments you tried. Please talk about things you tried but didn't work, or things you thought of doing but didn't. Finally, discuss how each group member contributed to the project. As usual, there is an expectation that every group member will have made some significant contribution to the project. 

## Submit to Kaggle 
rubric={accuracy:2}

Run your system over the test data, and submit the result (in the same format as the train/dev answers) to the Kaggle competition. The competition is hosted [here](https://www.kaggle.com/t/71ccaf26071b4bfc864251a03c204b4f). To get full points, you need to beat the public baseline. Use your capstone partner as your team name please!


In [ ]:
pred_test_slots = []
for inputs, labels, iobs, masks in tqdm(test_dataloader, total=len(test_dataloader)):
    out = model.decode(inputs.to(DEVICE), masks.to(DEVICE))
    pred_test_slots.extend(out)

assert len(pred_test_slots) == len(test_iter)

100%|██████████| 25/25 [00:05<00:00,  4.18it/s]


In [ ]:
HEADER = 'ID,Expected'

with open("kaggle_tags.csv", "w") as f:
    f.write(HEADER + '\n')
    for i, pred in enumerate(pred_test_slots):
        f.write(str(i) + "," + pred + "\n")

## Exercise: Kaggle competition (Optional)
rubric={raw:2}

As a team, compete to get the best result in the task. Since there are only 8 teams, the distribution of marks is a bit different than usual, only the top 3 groups will get bonus points. As usual, the rankings will be based on the score on the private leaderboard:


- 1st place: 2
- 2nd place: 1
- 3rd place: 0.5